In [ ]:
import pymysql
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from pyquery import PyQuery as pq
from urllib.parse import quote
import re
import json
import time

browser=webdriver.Chrome()
wait=WebDriverWait(browser,10)

base_url='https://space.bilibili.com/ajax/member/GetInfo?'
pattern=re.compile('.*?src="(.*)@.*',re.S)

f_count=0
m_count=0
u_count=0
def index_page(page):
    print("正在爬取第 ",str(page+1),' 页。')
    browser.get('https://space.bilibili.com/632887/#/fans/fans')
    if page>=1:
        submit=browser.find_element(By.CSS_SELECTOR,'.be-pager-next')
        submit.click()
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'.follow-main .relation-list .list-item')))
    get_fans_page()

def get_fans_page():
    html=browser.page_source
    doc=pq(html)
    items=doc('#page-follows .follow-main .relation-list .list-item').items()
    for item in items:
        html=item.find('.cover').html()
        image=re.findall(pattern,html)
        image_url=''
        if len(image) >=1 :
            image_url='https:'+image[0]
            
        
        info={
            'main_page':'https:'+item.find('.cover').attr('href'),
            'image':image_url,
            'name':item.find('.fans-name').text()
        }
#         print(info)
        id=re.search(r'm/(\d+)/',info.get('main_page'),re.S).group(1)
        get_fans_info(id)

def get_fans_info(id):
    global f_count,m_count,u_count
    request_headers  =   {
    'Host':'space.bilibili.com',
    'Referer':'https://space.bilibili.com/632887/',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1 Safari/605.1.15',
    'X-Requested-With':'XMLHttpRequest',
}
    payload={
        'mid':id,
        'csrf':'',
    }
    
#     {'status': True, 'data': {'mid': 632887, 'name': '伢伢gagako', 'sex': '女', 'rank': 10000, 
#                               'face': 'http://i1.hdslb.com/bfs/face/59816e144871157b404661a8149182f8b9b37269.jpg', 
#                               'regtime': 1352624241, 'spacesta': 0, 'birthday': '10-31',
#                               'sign': '一个正在成长的舞见QUQ 微博：伢伢同学 http://weibo.com/u/3197187502
#                               QQ群：146140306', 'level_info': {'current_level': 6}, 
#                               'official_verify': {'type': -1, 'desc': '', 'suffix': ''}, 
#                               'vip': {'vipType': 2, 'vipStatus': 1},
#                               'toutu': 'bfs/space/c9dae917e24b4fc17c4d544caf6b6c0b17f8692b.jpg', 
#                               'toutuId': 3, 'theme': 'default', 'theme_preview': '',
#                               'coins': 0, 'im9_sign': 'ce3cab374c16d5b0ca38220950209b36',
#                               'fans_badge': True}}
    response=requests.post('https://space.bilibili.com/ajax/member/GetInfo',data=payload,headers=request_headers)
    data=response.json()
    if data:
        detail={}
        detail['id']=data['data']['mid']
        detail['name']=data['data']['name']
        detail['sex']=data['data']['sex']
        detail['regtime']=timestamp2datetime(data['data']['regtime'])
        detail['birthday']=data['data']['birthday']
        detail['sign']=data['data']['sign']
        detail['level']=data['data']['level_info'].get('current_level')
        detail['vip']=data['data']['vip'].get('vipType')
        detail['coins']=data['data']['coins']
        detail['face']=data['data']['face']
#         以下将数据保存文件
#         with open ('/Users/limuyle/Desktop/webscrapy/yayadefans.txt','a',encoding='utf-8') as f:
#             for key in detail.keys():
#                 f.write(key+':  '+str(detail[key])+'\t')
#             f.write('\n'*2)
#             以下统计用户信息
        if detail['sex'] == '男':
            m_count+=1
        elif detail['sex'] == '女':
            f_count+=1
        else:
            u_count+=1
                
                
        
        
        
        
def timestamp2datetime(timestamp):
    format='%Y-%m-%d  %H:%M:%S'
    local_time=time.localtime(timestamp)
    datetime=time.strftime(format,local_time)
    return datetime    
    
    
    
        
def main():
    for i in range(0,5):
        index_page(i)
    browser.close()
    
if __name__ == '__main__':
    main()
    print('man fans: ',m_count)
    print('woman fans: ',f_count)
    print('unknown fans: ',u_count)
    print('total fans: ',m_count+f_count+u_count)
    print('rate: ',float(m_count)/(m_count+f_count+u_count))
        
    

        

In [5]:
import requests
from urllib.parse import urlencode
from pyquery import PyQuery as pq


base_url='https://space.bilibili.com/ajax/member/GetInfo'

headers={
    'Host':'space.bilibili.com',
    'Referer':'https://space.bilibili.com/632887/',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1 Safari/605.1.15',
    'X-Requested-With':'XMLHttpRequest',
}

def get_user_info(id):
    payload={
        'mid':id,
        'csrf':'',
    }
    response=requests.post(base_url,data=payload,headers=headers)
    print(response.json())
    
    
if __name__ == '__main__':
    get_user_info(632887)

{'status': True, 'data': {'mid': 632887, 'name': '伢伢gagako', 'sex': '女', 'rank': 10000, 'face': 'http://i1.hdslb.com/bfs/face/59816e144871157b404661a8149182f8b9b37269.jpg', 'regtime': 1352624241, 'spacesta': 0, 'birthday': '10-31', 'sign': '一个正在成长的舞见QUQ 微博：伢伢同学 http://weibo.com/u/3197187502 QQ群：146140306', 'level_info': {'current_level': 6}, 'official_verify': {'type': -1, 'desc': '', 'suffix': ''}, 'vip': {'vipType': 2, 'vipStatus': 1}, 'toutu': 'bfs/space/c9dae917e24b4fc17c4d544caf6b6c0b17f8692b.jpg', 'toutuId': 3, 'theme': 'default', 'theme_preview': '', 'coins': 0, 'im9_sign': 'ce3cab374c16d5b0ca38220950209b36', 'fans_badge': True}}
